In [143]:
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier

In [3]:
data = pd.read_csv("full_df.csv")

The following code is for the vectorization of ASJP code. Each index within the vector represents the relative frequency of that term within the word.

In [117]:
ASJPToIndex = {"i" : 0,
                "e" : 1,
                "E" : 2,
                "3" : 3,
                "a" : 4,
                "u" : 5,
                "o" : 6,
                "p" : 7,
                "b" : 8,
                "m" : 9,
                "f" : 10,
                "v" : 11,
                "8" : 12,
                "4" : 13,
                "t" : 14,
                "d" : 15,
                "s" : 16,
                "z" : 17,
                "c" : 18,
                "n" : 19,
                "S" : 20,
                "Z" : 21,
                "C" : 22,
                "j" : 23,
                "T" : 24,
                "5" : 25,
                "k" : 26,
                "g" : 27,
                "x" : 28,
                "N" : 29,
                "q" : 30,
                "G" : 31,
                "X" : 32,
                "7" : 33,
                "h" : 34,
                "l" : 35,
                "L" : 36,
                "w" : 37,
                "y" : 38,
                "r" : 39,
                "!" : 40}
#This method converts the given word to its vector form

vectors = pd.DataFrame(columns=range(0,40))
def getVector(string):
    ser = pd.Series(np.zeros(41), index=range(0,41))
    ser[41] = string.index[0]
    for c in string[0]:
        if c in ASJPToIndex:
            ser[ASJPToIndex[c]] += 1/len(string[0])
    global vectors
    vectors = vectors.append(ser,ignore_index=True)
#The following lines show example use
#bob = "aoelag"
#dave = 'xalk"ok"il'
#
#getVector(bob)
#vectors
#getVector(dave)
#vectors#.to_csv('bob.csv', sep=',', encoding='utf-8') #To file

In [125]:
#data.bark.dropna().apply(getVector) # cleaner but seems to obscure the column name
y = 0
def getget(x):
    global y
    print(x)#x.name)
pd.DataFrame(data, columns=['bird']).dropna().apply(getVector, axis=1)
vectors
#data.dropna().loc[:,"I":].transpose().apply(getget, axis=1)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,...,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,one
1,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,one
2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.333333,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,one
3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,one
4,0.000000,0.000000,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,one
5,0.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,...,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.0,one
6,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.500000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,one
7,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,one
8,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.666667,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,one
9,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.0,one


In [138]:
features = list(range(0,41))
X = vectors[features]
Y = vectors[41]


In [139]:
from sklearn.cross_validation import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, random_state=0)

C:\Users\Nicos\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [146]:
rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [147]:
print(rf.feature_importances_)

[0.00836055 0.02185545 0.0272866  0.         0.         0.05096839
 0.11129239 0.         0.22012193 0.12967837 0.         0.04726697
 0.         0.         0.         0.05075227 0.         0.
 0.         0.         0.         0.00801074 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.02602098
 0.00927638 0.0202127  0.         0.2688963  0.        ]
